In [8]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from scipy.io import loadmat
from matplotlib.ticker import MaxNLocator

In [9]:
# Define fonts and fontsize for plotting
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
fontsize = 15

In [10]:
# Define necessary functions
# Function to get displacement and void data
def get_data(file_directory, void_number):

    # Load the displacement data
    training_displacement_data = loadmat(os.path.join(file_directory, f'void_{void_number}_training_displacement_data_normalized.mat'))['displacement_data']
    validation_displacement_data = loadmat(os.path.join(file_directory, f'void_{void_number}_validation_displacement_data_normalized.mat'))['displacement_data']

    # Load the void data
    training_void_data = loadmat(os.path.join(file_directory, f'void_{void_number}_training_void_data.mat'))['void_data']
    validation_void_data = loadmat(os.path.join(file_directory, f'void_{void_number}_validation_void_data.mat'))['void_data']

    return training_displacement_data, validation_displacement_data, training_void_data, validation_void_data

In [11]:
# Residual class
class RESIDUAL():

    def __init__(self, input_shape, output_shape):
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        # Initialize input_layer here
        self.input_layer = None  

    # Method to build the hidden layers
    def build_hidden_layers(self):
        
        # Convolutional Layers
        # First Convolutional Layer
        x1 = tf.keras.layers.Conv1D(filters=180, kernel_size=10, padding='same', activation='relu', kernel_initializer = 'glorot_normal')(self.input_layer)
        x1 = tf.keras.layers.BatchNormalization()(x1)

        # Second Convolutional Layer
        x2 = tf.keras.layers.Conv1D(filters=180, kernel_size=10, padding='same', activation='relu', kernel_initializer = 'glorot_normal')(x1)
        x2 = tf.keras.layers.BatchNormalization()(x2)
        
        # Skip concatenated connection
        x2 = tf.keras.layers.Concatenate(axis=-1)([x1, x2])

        return x2

    # Method to build the overall model
    def build_model(self):
        
        # Input layer
        self.input_layer = tf.keras.layers.Input(shape=self.input_shape)

        # Hidden layer
        hidden_layer = self.build_hidden_layers()

        # Add a flatten layer
        flatten_layer = tf.keras.layers.Flatten()(hidden_layer)

        # Output Layer
        output_layer = tf.keras.layers.Dense(units=self.output_shape, activation='sigmoid')(flatten_layer)

        # Build model
        self.model = tf.keras.models.Model(inputs=[self.input_layer], outputs=[output_layer])

        return self.model

    # Method to compile the model
    def compile(self, optimizer, loss, evaluation_metric):
        
        # Compile model
        self.model.compile(optimizer=optimizer, loss=loss, metrics=evaluation_metric)

        return self.model
    
    # Define method to train the model
    def train(self, x_train, y_train, x_val, y_val, epochs, batch_size, callbacks):
        
        # Train model
        self.history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks,
                                      validation_data=(x_val, y_val))
        
        return self.history
    
    # Method to print summary of model
    def summary(self):
        
        self.model.summary()

In [12]:
class plots:
    
    def __init__(self, history, file_directory):

        self.history = history
        self.file_directory = file_directory

    def loss(self):

        loss_name = list(self.history.history.keys())[0]

        # Training
        loss = self.history.history[loss_name]
        val_loss = self.history.history['val_' + loss_name]

        loss_plot = plt.figure()
        epochs = range(1, len(loss)+1)
        plt.plot(epochs, loss, 'bo--', label = 'Training Loss', markersize = 2)
        plt.plot(epochs, val_loss, 'go--', label = 'Validation Loss', markersize = 2)
        plt.title('Training and Validation Loss', fontsize=fontsize)
        plt.xlabel('Epochs', fontsize=fontsize)
        plt.ylabel('Loss', fontsize=fontsize)
        plt.legend(['Training Loss', 'Validation Loss'], fontsize=fontsize)
        ax = loss_plot.gca()
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.savefig(self.file_directory + '/loss.pdf', bbox_inches='tight')
        
        return loss_plot

    def evaluation_metric(self):

        metric_name = list(self.history.history.keys())[1]
        
        # Training
        metric = self.history.history[metric_name]
        val_metric = self.history.history['val_' + metric_name]

        metric_plot = plt.figure()
        epochs = range(1, len(metric)+1)
        plt.plot(epochs, metric, 'bo--', label = 'Training Metric', markersize = 2)
        plt.plot(epochs, val_metric, 'go--', label = 'Validation Metric', markersize = 2)
        plt.title('Training and Validation Evaluation Metric', fontsize=fontsize)
        plt.xlabel('Epochs', fontsize=fontsize)
        plt.ylabel('Evaluation Metric', fontsize=fontsize)
        plt.legend(['Training Metric', 'Validation Metric'], fontsize=fontsize)
        ax = metric_plot.gca()
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.savefig(self.file_directory + '/evaluation_metric.pdf', bbox_inches='tight')

        return metric_plot

In [13]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'residual-concatenate', 'training_results')

In [14]:
# Get the normalized data for all three voids
void_0_training_displacement_data, void_0_validation_displacement_data, void_0_training_void_data, void_0_validation_void_data = get_data(normalized_data_directory, 0)
void_1_training_displacement_data, void_1_validation_displacement_data, void_1_training_void_data, void_1_validation_void_data = get_data(normalized_data_directory, 1)
void_2_training_displacement_data, void_2_validation_displacement_data, void_2_training_void_data, void_2_validation_void_data = get_data(normalized_data_directory, 2)

In [ ]:
# Vertically stack the training, validation and test data
# Displacement
training_displacement_data = np.vstack((void_0_training_displacement_data, void_1_training_displacement_data, void_2_training_displacement_data))
validation_displacement_data = np.vstack((void_0_validation_displacement_data, void_1_validation_displacement_data, void_2_validation_displacement_data))

# Void
training_void_data = np.vstack((void_0_training_void_data, void_1_training_void_data, void_2_training_void_data))
validation_void_data = np.vstack((void_0_validation_void_data, void_1_validation_void_data, void_2_validation_void_data))

In [ ]:
# Randomize the dataset using the same seed
np.random.seed(42)

# Randomize the training data
random_indices = np.random.permutation(training_displacement_data.shape[0])
training_displacement_data = training_displacement_data[random_indices]
training_void_data = training_void_data[random_indices]

# Randomize the validation data
random_indices = np.random.permutation(validation_displacement_data.shape[0])
validation_displacement_data = validation_displacement_data[random_indices]
validation_void_data = validation_void_data[random_indices]

In [ ]:
# Define variables that remain constant during the training
input_shape = void_0_training_displacement_data.shape[1:]
output_shape = void_0_training_void_data.shape[1]

In [ ]:
# Create an instance of the residual class
model = RESIDUAL(input_shape, output_shape)

In [ ]:
# Build and the model
model.build_model()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy', evaluation_metric = tf.metrics.Precision(name='precision'))

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Define callbacks
# Early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights=True)

In [ ]:
# Train the model
print('Training the model...')
history = model.train(training_displacement_data, training_void_data, 
                      validation_displacement_data, validation_void_data,
                      epochs = 1000, 
                      batch_size = 32,
                      callbacks=[early_stopping_callback])

In [ ]:
plot = plots(history, trained_results_directory)
loss_plot = plot.loss()
evaluation_metric_plot = plot.evaluation_metric()

In [ ]:
# Save the model
print('Saving the model...')
model.model.save(os.path.join(trained_results_directory, 'model.h5'))